<!-- https://www.kaggle.com/code/iakhtar0/63-next-word-predictor-lstm-campusx -->

In [18]:
import sys
import platform
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import nltk
import sklearn

print("🔧 Environment Information")
print("=" * 40)
print(f"Python version      : {platform.python_version()}")
print(f"TensorFlow version  : {tf.__version__}")
print(f"NumPy version       : {np.__version__}")
print(f"Pandas version      : {pd.__version__}")
print(f"Matplotlib version  : {matplotlib.__version__}")
print(f"Seaborn version     : {sns.__version__}")
print(f"NLTK version        : {nltk.__version__}")
print(f"scikit-learn version: {sklearn.__version__}")
print("=" * 40)

# Optional: GPU info
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU detected: {gpus[0].name}")
else:
    print("⚠️ No GPU detected (running on CPU).")


🔧 Environment Information
Python version      : 3.12.3
TensorFlow version  : 2.20.0
NumPy version       : 2.3.4
Pandas version      : 2.3.3
Matplotlib version  : 3.10.7
Seaborn version     : 0.13.2
NLTK version        : 3.9.2
scikit-learn version: 1.7.2
✅ GPU detected: /physical_device:GPU:0


In [19]:
# Deep Learning library
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.layers import Conv1D, Dropout, MaxPooling1D
from tensorflow.keras import backend as K
print(tf.__version__)

2.20.0


In [20]:
from pathlib import Path
NOTEBOOK_DIR  = Path().resolve()
BASE_DIR = NOTEBOOK_DIR.parents[0]
DATASET_DIR = BASE_DIR /  "data" / 'Twitter' / "preprocessed"
DATASET_FILE_PATH = DATASET_DIR  / 'train.csv'
print(DATASET_FILE_PATH.exists())
DATASET_FILE_PATH


True


PosixPath('/home/srirama/sr_proj/EmotionAnalysis/data/Twitter/preprocessed/train.csv')

In [21]:
df = pd.read_csv(DATASET_FILE_PATH,usecols=["sentiment","cleaned_text"])
df["cleaned_text"] = df["cleaned_text"].astype(str)
df.isnull().sum()

sentiment       0
cleaned_text    0
dtype: int64

In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [23]:
tokenizer = Tokenizer(oov_token="None")
# Gives ID to each word in the sentences (entire column)
tokenizer.fit_on_texts(df['cleaned_text'])
vocab_size  = len(tokenizer.word_index) # 31774 unique words
print("Total unique words : ", vocab_size )
# print("Unique words with indices",tokenizer.word_index)
sequences  = tokenizer.texts_to_sequences(df['cleaned_text'])
print("Total Sequences",len(sequences) )# = total rows
print("Length of 1st sequence",len(sequences[0])) # = 1st row word count
sequences[:4]

Total unique words :  22499
Total Sequences 69490
Length of 1st sequence 14


[[3, 71, 360, 4, 2, 6267, 7, 6, 5, 3, 55, 384, 15, 25],
 [3, 71, 153, 17, 160, 7, 6, 5, 3, 55, 384, 15, 25],
 [3, 71, 360, 17, 160, 7, 6, 5, 3, 55, 1603, 15, 25],
 [3, 71, 153, 17, 160, 7, 6, 5, 3, 55, 1603, 15, 24, 25]]

In [24]:
df.head(5)

,sentiment,cleaned_text
0,Positive,i am coming to the borders any day now i wil k...
1,Positive,i am geting on borderlands any day now i wil k...
2,Positive,i am coming on borderlands any day now i wil m...
3,Positive,i am geting on borderlands any day now i wil m...
4,Positive,i am geting into borderlands any day now i can...


In [25]:
maxlen = 75
padded_sentence_sequences = pad_sequences(sequences, maxlen = maxlen, padding='pre')
print("Padded Sequence shape :",padded_sentence_sequences.shape)
padded_sentence_sequences

Padded Sequence shape : (69490, 75)


array([[   0,    0,    0, ...,  384,   15,   25],
       [   0,    0,    0, ...,  384,   15,   25],
       [   0,    0,    0, ..., 1603,   15,   25],
       ...,
       [   0,    0,    0, ...,   91,  269, 1902],
       [   0,    0,    0, ...,  132,  269, 1902],
       [   0,    0,    0, ...,    3,  269, 1902]],
      shape=(69490, 75), dtype=int32)

In [26]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df['sentiment'])
y = to_categorical(y_encoded)  # one-hot encode for multi-class

In [27]:
from keras.models import Sequential
from keras.layers import SimpleRNN , Dense , Embedding, LSTM, GRU 
import mlflow
import mlflow.tensorflow
import warnings
warnings.filterwarnings("ignore")

In [28]:
tracking_path = f"http://localhost:5000/"
mlflow.set_tracking_uri(tracking_path)
print("Tracking URI:", mlflow.get_tracking_uri())
mlflow.set_experiment("1-KERAS-TUNER")

2025/11/19 15:57:29 INFO mlflow.tracking.fluent: Experiment with name '1-KERAS-TUNER' does not exist. Creating a new experiment.


Tracking URI: http://localhost:5000/


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1763567849174, experiment_id='1', last_update_time=1763567849174, lifecycle_stage='active', name='1-KERAS-TUNER', tags={}>

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(padded_sentence_sequences, y, test_size=0.4, random_state=42)
TIMESTEP = maxlen
INPUT_NEURONS = 1
NUM_CLASSES = y.shape[1]

In [30]:
import keras_tuner as kt
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [46]:
class CustomHyperModel(kt.HyperModel):
    TRIAL_ID : int = 0
    def build(self,hp):
        hyperParameters = {
            "simple_rnn_units" : hp.Choice('units', [8, 16, 32]),
            "learning_rate" : hp.Choice('lr', [1e-3, 1e-4]),
            "dense_1_units" :hp.Int('dense_1_units', min_value=32, max_value=64, step=4),
            "dense_2_units" :hp.Int('dense_2_units', min_value=16, max_value=64, step=8),
            "dense_3_units" :hp.Int('dense_3_units', min_value=8, max_value=64, step=8)


        }
        model = Sequential()
        model.add(SimpleRNN(hyperParameters["simple_rnn_units"], input_shape=(TIMESTEP ,INPUT_NEURONS),return_sequences=False))
        # Dense layers (tunable)
        model.add(Dense(hyperParameters["dense_1_units"], activation='relu'))
        model.add(Dense(hyperParameters["dense_2_units"], activation='relu'))
        model.add(Dense(hyperParameters["dense_3_units"], activation='relu'))
        model.add(Dense(NUM_CLASSES, activation='softmax'))
        
        model.compile(
            loss='categorical_crossentropy',
            optimizer=tf.keras.optimizers.Adam(learning_rate = hyperParameters["learning_rate"]),
            metrics=['accuracy']
        )
        return model
    def fit(self, hp, model, *args, **kwargs):
        CustomHyperModel.TRIAL_ID+=1
        with mlflow.start_run(run_name=f"trial_{CustomHyperModel.TRIAL_ID}"):
            mlflow.log_params(hp.values)
            mlflow.tensorflow.autolog()
            return model.fit(*args , **kwargs)
        

In [47]:
tuner = kt.Hyperband(
    hypermodel = CustomHyperModel(),
    objective = 'val_loss',
    max_epochs = 20,
    overwrite=True,
    directory="/tmp/tb",
)

In [48]:
tuner.search(
    X_train, y_train, 
    epochs = 10, 
    validation_data=(X_val, y_val),
    batch_size=32,
    # callbacks = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
)

Trial 24 Complete [00h 05m 06s]
val_loss: 1.3676881790161133

Best val_loss So Far: 1.3497672080993652
Total elapsed time: 01h 41m 26s

Search: Running Trial #25

Value             |Best Value So Far |Hyperparameter
32                |32                |units
0.001             |0.0001            |lr
40                |64                |dense_1_units
24                |40                |dense_2_units
64                |24                |dense_3_units
20                |20                |tuner/epochs
7                 |7                 |tuner/initial_epoch
1                 |2                 |tuner/bracket
1                 |2                 |tuner/round
0019              |0012              |tuner/trial_id



Epoch 8/20
 184/1303 ━━━━━━━━━━━━━━━━━━━━ 30s 27ms/step - accuracy: 0.3207 - loss: 1.3618🏃 View run trial_25 at: http://localhost:5000/#/experiments/1/runs/bff8ad08316b42f98811113548f16309
🧪 View experiment at: http://localhost:5000/#/experiments/1


KeyboardInterrupt: 

In [49]:
tuner.results_summary()

Results summary
Results in /tmp/tb/untitled_project
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 0016 summary
Hyperparameters:
units: 32
lr: 0.0001
dense_1_units: 64
dense_2_units: 40
dense_3_units: 24
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0012
Score: 1.3497672080993652

Trial 0017 summary
Hyperparameters:
units: 32
lr: 0.0001
dense_1_units: 40
dense_2_units: 56
dense_3_units: 64
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0013
Score: 1.3538638353347778

Trial 0012 summary
Hyperparameters:
units: 32
lr: 0.0001
dense_1_units: 64
dense_2_units: 40
dense_3_units: 24
tuner/epochs: 7
tuner/initial_epoch: 3
tuner/bracket: 2
tuner/round: 1
tuner/trial_id: 0008
Score: 1.3591728210449219

Trial 0013 summary
Hyperparameters:
units: 32
lr: 0.0001
dense_1_units: 40
dense_2_units: 56
dense_3_units: 64
tuner/epochs: 7
tuner/initial_epoch: 3
tuner/bracket: 2
tuner/round: 1
tuner/

In [50]:
best_model = tuner.get_best_models()[0]
best_hyperparameters= tuner.get_best_hyperparameters()[0].values
best_hyperparameters

{'units': 32,
 'lr': 0.0001,
 'dense_1_units': 64,
 'dense_2_units': 40,
 'dense_3_units': 24,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0012'}

In [52]:
best_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 32)             │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 40)             │         2,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 24)             │           984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           100 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,884 (26.89 KB)

 Trainable params: 6,884 (26.89 KB)

 Non-trainable params: 0 (0.00 B)

In [53]:
best_model.save("best_model.keras")

In [54]:
model = tf.keras.models.load_model("best_model.keras")

In [56]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 32)             │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 40)             │         2,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 24)             │           984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           100 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,654 (80.68 KB)

 Trainable params: 6,884 (26.89 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 13,770 (53.79 KB)